# 필요한 패키지, 모듈 및 함수 불러오기

In [1]:
import requests, time, sqlite3, os
import pandas as pd

# 로그인 함수와 원본 이미지 추출 함수

In [2]:
def login(username, password):
    """인스타그램 로그인 함수입니다. 쿠키값을 반환합니다."""
    login_url = 'https://www.instagram.com/accounts/login/ajax/'

    payload = {
        'username': username,
        'enc_password': f'#PWD_INSTAGRAM_BROWSER:0:{int(time.time())}:{password}',
        'queryParams': '{}',
        'optIntoOneTap': 'false',
        'stopDeletionNonce': '',
        'trustedDeviceRecords': '{}',
    }

    with requests.Session() as s:
        csrftoken = requests.get('https://www.instagram.com/accounts/login/').cookies['csrftoken']
        r = s.post(login_url, data=payload, headers={
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36',
            'x-requested-with': 'XMLHttpRequest',
            'referer': 'https://www.instagram.com/',
            'x-csrftoken': csrftoken
        })
    
    cookies = r.cookies.get_dict()
    
    # 로그인 성공여부 확인
    if cookies.get('ds_user_id'):
        print(True)
    else:
        print(False)
        
    return cookies

def get_origin_image(media):
    """media에서 원본 사진을 가져옵니다. 리스트를 반환합니다."""
    origin_images = []
    if media.get('carousel_media'):
        for carousel_media in media['carousel_media']:
            if media.get('video_versions'):
                continue    # 영상은 스킵한다.
                # video_versions = media['video_versions']
                # origin_video = sorted(candidates,
                #                       key=lambda video_version: video_version['width'],
                #                       reverse=True)[0]['url']
            candidates = carousel_media['image_versions2']['candidates']
            origin_image = sorted(candidates, key=lambda candidate: candidate['width'], reverse=True)[0]['url']
            origin_images.append([carousel_media['id'], media['taken_at'], origin_image])
                    
    else:
        if media.get('video_versions'):
            return origin_images
        candidates = media['image_versions2']['candidates']
        origin_image = sorted(candidates, key=lambda candidate: candidate['width'], reverse=True)[0]['url']
        origin_images.append([media['id'], media['taken_at'], origin_image])
    return origin_images

def mkdirs(path_dir):
    if not os.path.exists(path_dir):
        os.makedirs(path_dir)

# 크롤링 시작

In [3]:
# 로그인
username = '{username}'
password = '{password}'

cookies = login(username=username, password=password)

True


In [10]:
cookies.keys()

dict_keys(['mid', 'ig_did', 'ig_nrcb', 'shbid', 'shbts', 'csrftoken', 'ds_user_id', 'sessionid', 'rur'])

In [5]:
# 필요한 cookies, headers 및 data 생성
cookies = {
    'mid': cookies['mid'],
    'ig_did': cookies['ig_did'],
    'ig_nrcb': '1',
    'shbid': '"13409\\05441319676062\\0541679302382:01f7c49ebc0fc5160226c32f4a46829699c5f87ead915c19014a1d988067088a2bdeede2"',
    'shbts': '"1647766382\\05441319676062\\0541679302382:01f7b1785c6fd4b9f04643b67728122f5ea806f0485f4bf90a8f07701f484e3b26b336c7"',
    'csrftoken': cookies['csrftoken'],
    'ds_user_id': cookies['ds_user_id'],
    'sessionid': cookies['sessionid'],
    'rur': cookies['rur'],
}

headers = {
    'authority': 'i.instagram.com',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="99", "Google Chrome";v="99"',
    'x-ig-app-id': '936619743392459',
    'x-ig-www-claim': 'hmac.AR2TYlo3TmJksgsgmVXXf1Z5plzoAT9NZk30ejeUuGm_MsFG',
    'sec-ch-ua-mobile': '?0',
    'x-instagram-ajax': 'fde055a3a0e4',
    'content-type': 'application/x-www-form-urlencoded',
    'accept': '*/*',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36',
    'x-asbd-id': '198387',
    'x-csrftoken': cookies['csrftoken'],
    'sec-ch-ua-platform': '"Windows"',
    'origin': 'https://www.instagram.com',
    'sec-fetch-site': 'same-site',
    'sec-fetch-mode': 'cors',
    'sec-fetch-dest': 'empty',
    'referer': 'https://www.instagram.com/',
    'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7,th;q=0.6',
    # Requests sorts cookies= alphabetically
    # 'cookie': 'mid=YNj0jQALAAFseBFmmfpRDzFSV_BT; ig_did=1996DD5D-2FB1-4BE9-ADB1-0EE33EDA47B1; ig_nrcb=1; shbid="13409\\05441319676062\\0541679302382:01f7c49ebc0fc5160226c32f4a46829699c5f87ead915c19014a1d988067088a2bdeede2"; shbts="1647766382\\05441319676062\\0541679302382:01f7b1785c6fd4b9f04643b67728122f5ea806f0485f4bf90a8f07701f484e3b26b336c7"; csrftoken=YjVqxSp9RmOmaia58YFD5cOCYamI1ul3; ds_user_id=41319676062; sessionid=41319676062%3AZWNhhePnUvAO4a%3A16; rur="NAO\\05441319676062\\0541679321348:01f7583980e3a91fc37873ce58fefcbc8e887c50a66d869f211fb7a99174bfbed73c5e8b"',
}

# 0페이지부터 크롤링
data = {
    'include_persistent': '0',
    # 'max_id': 'QVFCVjNHdk01dDhVTkRYcWlueERYOUszUGtobVdwV0R1NE5lSzk2ZEl1SGEyOWNzLUdDVk5kSE50YlF4M0RJMmRrOFZmem8wcHRyNExwT0szekdmalhMMw==',
    # 'page': 1,
    'surface': 'grid',
    'tab': 'recent',
}

In [6]:
# 태그로 크롤링
tag = 'welshcorgi'
data['page'] = 0
data['max_id'] = ''

In [7]:
num_posts = 100 # 필요한 이미지 개수
images = []

while len(images) <= num_posts:
    print(f'\r{len(images)} / {num_posts}', end='')
    res = requests.post(f'https://i.instagram.com/api/v1/tags/{tag}/sections/',
                        headers=headers, cookies=cookies, data=data)
    json_data = res.json()
    
    sections = json_data['sections']

    for section in sections:
        medias = section['layout_content']['medias']
        for media in medias:
            images += get_origin_image(media['media'])

    data['max_id'] = json_data['next_max_id']
    data['page'] = json_data['next_page']
    
    time.sleep(2)

print(f'\r{len(images)} / {num_posts}', end='')

162 / 100

In [9]:
len(images)

146

In [108]:
next_max_id = json_data['next_max_id']
columns = ['media_id', 'timestamp', 'url']
df = pd.DataFrame(data=images, columns=columns)
df

,media_id,timestamp,url
0,2798263917856992275_2077759989,1647799076,https://scontent-ssn1-1.cdninstagram.com/v/t51...
1,2798263917840255443_2077759989,1647799076,https://scontent-ssn1-1.cdninstagram.com/v/t51...
2,2798263917655718851_2077759989,1647799076,https://scontent-ssn1-1.cdninstagram.com/v/t51...
3,2798262518419839486_36105660916,1647798908,https://scontent-ssn1-1.cdninstagram.com/v/t51...
4,2798262518444999986_36105660916,1647798908,https://scontent-ssn1-1.cdninstagram.com/v/t51...
...,...,...,...
1054,2797991437194962381_1751467914,1647766609,https://scontent-ssn1-1.cdninstagram.com/v/t51...
1055,2797991514185475961_1751467914,1647766609,https://scontent-ssn1-1.cdninstagram.com/v/t51...
1056,2797991572966110057_1751467914,1647766609,https://scontent-ssn1-1.cdninstagram.com/v/t51...
1057,2797991572664101855_1751467914,1647766609,https://scontent-ssn1-1.cdninstagram.com/v/t51...


# database에 저장

In [109]:
# next_max_id 데이터 추가
with sqlite3.connect('instagram.db') as conn:
    table = 'next_max_id'
    cur = conn.cursor()
    sql = f"""CREATE TABLE IF NOT EXISTS {table} (
            name TEXT PRIMARY KEY,
            media_id TEXT
        );"""
    cur.execute(sql)
    
    sql = f"""REPLACE INTO {table} (name, media_id) VALUES (?, ?);"""
    cur.execute(sql, (tag, next_max_id))

In [110]:
with sqlite3.connect('instagram.db') as conn:
    df.to_sql(name=tag,             # 테이블 이름
              con=conn,             # connection 객체
              index=False,          # index 무시
              if_exists='replace',  # 이미 존재하는 테이블 삭제 후 추가
              chunksize=1000,       # 해당 행 개수만큼 끊어서 데이터 추가
              )

# database에서 불러오기

In [116]:
with sqlite3.connect('instagram.db') as conn:
    df_from_db = pd.read_sql(f'SELECT * FROM {tag}', con=conn, index_col='media_id')

In [117]:
df_from_db

,timestamp,url
media_id,,
2798263917856992275_2077759989,1647799076,https://scontent-ssn1-1.cdninstagram.com/v/t51...
2798263917840255443_2077759989,1647799076,https://scontent-ssn1-1.cdninstagram.com/v/t51...
2798263917655718851_2077759989,1647799076,https://scontent-ssn1-1.cdninstagram.com/v/t51...
2798262518419839486_36105660916,1647798908,https://scontent-ssn1-1.cdninstagram.com/v/t51...
2798262518444999986_36105660916,1647798908,https://scontent-ssn1-1.cdninstagram.com/v/t51...
...,...,...
2797991437194962381_1751467914,1647766609,https://scontent-ssn1-1.cdninstagram.com/v/t51...
2797991514185475961_1751467914,1647766609,https://scontent-ssn1-1.cdninstagram.com/v/t51...
2797991572966110057_1751467914,1647766609,https://scontent-ssn1-1.cdninstagram.com/v/t51...


# 크롤링한 이미지 저장

In [143]:
path_to_dataset = r'C:\project\Machine Learning Project\yolov3\tf2-keras-yolo3-master\견종_10종\test'


for i in range(len(df_from_db))[:10]:
    url = df_from_db.iloc[i].url
    media_id = df_from_db.iloc[i].name
    timestamp = df_from_db.iloc[i].timestamp
    time_str = time.strftime('%Y-%m-%d %H.%M.%S', time.localtime(int(timestamp)))

    filename = f"{path_to_dataset}/#{tag}/{time_str} {media_id}.jpg"
    mkdirs(os.path.dirname(filename))
    
    chunk_size = 1024
    start = 0
    with requests.Session().get(url, stream=True) as r:
        with open(filename, 'wb') as fp:
            for chunk in r.iter_content(chunk_size=chunk_size):
                fp.seek(start)
                fp.write(chunk)
                start += chunk_size